# Regex in Pandas

**Inhalt:** Regular Expressions in Pandas anwenden

**Nötige Skills:** Regex in Python

**Lernziele:**
- Ein praktisches Beispiel kennenlernen, wo Regex nützlich sein kann

# Das Beispiel

Das Bundesamt für Statistik stellt oft Dateien in verknorkster Form zur Verfügung, zum Beispiel, wenn man Auswertungen nach Gemeinden, Bezirken und Kantonen über das interaktive Portal generieren lässt: https://www.pxweb.bfs.admin.ch/pxweb/de/

Das vorliegende Beispiel beinhaltet die Bevölkerungszahlen, gegliedert nach Zivilstand einerseits (ledig, verheiratet, etc) und nach räumlicher Struktur (Gemeinden, Bezirke, Kantone) andererseits.

Das File ist gespeichert unter `dataprojects/BFS/px-x-0102010000_103.xlsx`

Unser 1. Ziel ist: Wir wollen die Tabelle filtern
- nur die Gemeinden
- nur die Bezirke
- nur die Kantone

Unser 2. Ziel ist: Wir wollen bei den Gemeinden die jeweilige Nummer isolieren

## Vorbereitung

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
%matplotlib inline

## Datei laden

In [ ]:
path = 'dataprojects/BFS/px-x-0102010000_103.xlsx'

In [ ]:
df = pd.read_excel(path)

## Explorieren

In [ ]:
df.head(20)

Welche Einheitstypen gibt es? Und welches Muster haben sie?
- Gemeinde ("...... 9999 Gemeindename")
- Bezirk (">> Bezirsname")
- Kantone ("- Kantonsname")
- Land ("Land")

## Pandas-Funktionen, die Regex brauchen können

Einige Befehle heissen leicht anders, funktionieren aber sehr ähnlich wie in der re.Library

- **`str.contains(r"regex")`**: das Pendant zu `re.search()` - ja/nein-Antwort

- **`str.extract(r"regex")`**: auch ähnlich wie `re.search()` - Suchergebnis als Antwort

- **`str.replace(r"regex", "str")`**: das Pendant zu `re.sub()` - ersetzt Match mit String

Wir wenden diese Funktionen jetzt an.

### Aber zuerst ...

**Quiz:** Zuerst brauchen wir die Regex-Ausdrücke, um die Einheiten zu erkennen.

Am besten mit Tests beginnen, ob die Regex an einer Einheit anschlägt - und jeweils auch testen, ob die Regex bei Einheiten, die wir *nicht* wollen, auch *nicht* anschlägt.

**Test für Kantone**

In [ ]:
# Unsere Regex
regex_k = 

In [ ]:
# Schlägt die regex bei den Kantonen an?
# (Gut ist, wenn wir ein "Match"-Objekt erhalten)
re.search(regex_k, "- Aargau")

In [ ]:
# Schlägt die regex bei den Bezirken NICHT an?
# (Gut ist hier, wenn wir nichts zurückerhalten)
re.search(regex_k, ">> Wahlkreis Luzern-Stadt")

In [ ]:
# Schlägt die regex bei den Gemeinden NICHT an?
re.search(regex_k, "......0001 Aeugst am Albis")

**Test für Bezirke**

In [ ]:
# Unsere Regex
regex_b = 

In [ ]:
# Schlägt die regex bei den Bezirken an?


In [ ]:
# Schlägt die regex bei den Kantonen NICHT an?


In [ ]:
# Schlägt die regex bei den Gemeinden NICHT an?


**Test für Gemeinden**

In [ ]:
# Unsere Regex


In [ ]:
# Test für Gemeinde


In [ ]:
# Schlägt die regex bei den Kantonen NICHT an?


In [ ]:
# Schlägt die regex bei den Bezirken NICHT an?


## 1. Daten filtern

Nachdem wir nun die passenden Regex-Ausdrücke gefunden haben, können wir unsere Daten filtern.

Ziel: Wir erstellen ein separates DF für Gemeinden, Bezirke, Kantone.

Hier können wir die Funktion `str.contains()` gut brauchen.

Wir testen damit mal, ob die Einträge in der Spalte "Einheit" eine Gemeinde sind:

In [ ]:
df['Einheit'].str.contains(regex_g)

Basierend auf dieser True/False-Liste können wir nun die Tabelle filtern.

In [ ]:
df[df['Einheit'].str.contains(regex_g)]

Wir speichern den Output in einer separaten DF (als Kopie, damit wir nachher keinen Ärger kriegen).

In [ ]:
df_g = df[df['Einheit'].str.contains(regex_g)].copy()

**Quiz:** Erstellen Sie zwei weitere Dataframes, in denen nur die Kantone und Bezirke sind.

In [ ]:
df_k = df[df['Einheit'].str.contains(regex_k)].copy()

In [ ]:
df_b = df[df['Einheit'].str.contains(regex_b)].copy()

## 2. Daten extrahieren

Wir wenden uns den Gemeinden zu. Unser Ziel: Die Gemeindenummer in eine separate Spalte ziehen.

In [ ]:
df_g.head(2)

Hier kommt die Funktion `str.extract()` gelegen. Um sie zu benutzen, müssen wir zwei Dinge wissen:
1. die Regex, die zu dem gesamten Ausdruck passt.
2. Den Teil der Regex, den wir in Klammern `()` setzen und extrahieren wollen.

In [ ]:
df_g['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")

Wir können den Output dieser Funktion für die neue Spalte setzen.

In [ ]:
df_g['Nummer'] = df_g['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")

In [ ]:
df_g.head(10)

**Quiz:** Erstellen Sie im Gemeinde-DF eine weitere Spalte ("Name"), die den Namen der Gemeinde enthält!

## Übung

### Teil 1

**Hier haben Sie Gelegenheit noch etwas mit den Regex-Funktionen in Pandas zu experimentieren.**

Wir arbeiten wieder mit dem Haupt-Dataframe am Anfang: `df`.

**Aufgabe 1:** Erstellen Sie im DF eine zusätzliche Spalte mit dem Namen "Typ". Darin soll stehen, ob es sich bei der betreffenden Zeile um eine Gemeinde, einen Bezirk, einen Kanton oder ein Land handelt.

Tipp: Arbeiten Sie mit `df.loc[]`

**Aufgabe 2:** Bereinigen Sie die Bezeichnungen in der Spalte "Einheit". Wir wollen keine Nummern und keine Sonderzeichen mehr darin sehen, nur noch die Namen der Gemeinden, Bezirke und Kantone.

Tipp: Arbeiten Sie mit `str.replace()`

In [ ]:
# zum Testen, ob es bei den Kantonen funktioniert hat:
df[df['Typ'] == 'Kanton'].head(10)

In [ ]:
# zum Testen, ob es bei den Bezirken funktioniert hat:
df[df['Typ'] == 'Bezirk'].head(10)

In [ ]:
# zum Testen, ob es bei den Gemeinden funktioniert hat:
df[df['Typ'] == 'Gemeinde'].head(10)

### Teil 2

**Hier können Sie nochmals etwas Pivoting und Plotting trainieren**

In [ ]:
df.to_csv('dataprojects/BFS/Einheiten_kategorisiert.csv', index=False)

Zur Sicherheit laden wir die Daten nochmals neu - mit dem Stand der Übung bis hier.

In [ ]:
df = pd.read_csv('dataprojects/BFS/Einheiten_kategorisiert.csv')

**Aufgabe 1:** Finden Sie die drei Gemeinden mit der höchsten Quote an Verwitweten Personen in der Bevölkerung.

Tipp zum Vorgehen:
1. DF nach Gemeinden filtern
1. DF nach Zivilstand pivotieren
1. Total der Bevölkerung mit `sum()` ausrechnen (`axis=1`)
1. Verwitweten-Quote ausrechnen
1. DF sortieren

In [ ]:
# DF nach gemeinden filtern und pivotieren


In [ ]:
# Bevölkerungstotal ausrechnen pro Gemeinde


In [ ]:
# Verwitweten-Quote ausrechnen


In [ ]:
# Sortieren und Top 3


**Aufgabe 2:** Welcher Kanton hat die höchste Ledigen-Quote?

In [ ]:
# DF nach Kantonen filtern und pivotieren


In [ ]:
# Bevölkerungstotal ausrechnen pro Kanton


In [ ]:
# Ledigen-Quote ausrechnen


In [ ]:
# Sortieren und obersten anzeigen


**Aufgabe 3:** Zeichnen Sie einen Bar-Chart mit der Verheirateten-Quote aller Kantone

Tipp: Benutzen Sie das pivotierte DF, das Sie in Aufgabe 2 erstellt haben.

In [ ]:
# Verheirateten-Quote ausrechnen


In [ ]:
#Sortieren und plotten


**Aufgabe 4:** Zeichnen Sie ein Stacked-Bar-Chart mit den relativen Zivilstands-Anteilen in zwei Gemeinden:
- Corippo
- Meienried

Tipp zu Vorgehen:
1. DF nach Gemeinden filtern und pivotieren
1. Total der Einwohner für jede Gemeinde (in separater Series ablegen)
1. Pivotiertes DF mit `div()` durch das Total dividieren (`axis=1`)
1. mit `.loc[]` die beiden Gemeinden wählen
1. Plot mit `stacked=True`

In [ ]:
# DF nach gemeinden filtern und pivotieren


In [ ]:
# Total der Einwohner für jede Gemeinde


In [ ]:
# Neues DF mit relativen Werten


In [ ]:
# Nur Meienried und Corippo anzeigen


In [ ]:
# Plot
